# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [42]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [ ]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

In [2]:
from nltk import word_tokenize
import pandas as pd

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

In [4]:
descriptions = pd.read_csv('preprocessed_descriptions.csv')

In [10]:
unique = set(i for i in word_tokenize(str(descriptions.preprocessed_descriptions)))

In [11]:
unique

{',',
 '...',
 '0',
 '1',
 '2',
 '29995',
 '29996',
 '29997',
 '29998',
 '29999',
 '3',
 '30000',
 '4',
 ':',
 'Length',
 'Name',
 'a',
 'an',
 'and',
 'ask',
 'at',
 'based',
 'but',
 'by',
 'cake',
 'casual',
 'changed',
 'chef',
 'children',
 'company',
 'cookies',
 'created',
 'delicious',
 'design',
 'din',
 'dtype',
 'early',
 'even',
 'fall',
 'family',
 'fondue',
 'for',
 'fou',
 'french',
 'fresh',
 'friends',
 'g',
 'go',
 'heard',
 'homem',
 'i',
 'is',
 'it',
 'ive',
 'late',
 'made',
 'me',
 'meska',
 'my',
 'object',
 'of',
 'on',
 'original',
 'originally',
 'plum',
 'preprocessed_descriptions',
 'recipe',
 'romantic',
 's',
 'scott',
 'sisterinlaw',
 'so',
 'soup',
 'summer',
 'surprised',
 'that',
 'the',
 'their',
 'these',
 'think',
 'this',
 'thought',
 'traditional',
 'us',
 'very',
 'were'}

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

In [12]:
from random import choices
from nltk.metrics.distance import edit_distance

for _ in range(5):
    sample = choices(list(unique), k=2)
    print(sample, edit_distance(*sample))

['4', 'chef'] 4
['fresh', 'cookies'] 6
['29996', '0'] 5
['the', '4'] 3
['29999', 'romantic'] 8


1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

In [16]:
with open('litw-win.txt', 'r') as fp:
    lines = fp.readlines()
words = [line.strip().split()[-1] for line in lines]

In [17]:
def nearest(word: str, k: int):
    return sorted(words, key=lambda x: edit_distance(word, x))[:k]


nearest('Стемминг', 3)

['термин', 'термина', 'термины']

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

In [20]:
import nltk

nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [23]:
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

In [21]:
stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

processed_words = pd.DataFrame(({
    'word': x,
    'stemmed_word': stemmer.stem(x),
    'normalized_word': lemmatizer.lemmatize(x)
} for x in unique
))

In [22]:
processed_words

,word,stemmed_word,normalized_word
0,cake,cake,cake
1,think,think,think
2,g,g,g
3,family,famili,family
4,29998,29998,29998
...,...,...,...
81,scott,scott,scott
82,me,me,me
83,but,but,but
84,1,1,1


2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [25]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [26]:
from nltk.corpus import stopwords

stopwrds = stopwords.words('english')

In [37]:
tokenized_ = [word_tokenize(str(i)) for i in descriptions.preprocessed_descriptions]
tokenized = [item for sublist in tokenized_ for item in sublist]
tokenized

['an',
 'original',
 'recipe',
 'created',
 'by',
 'chef',
 'scott',
 'meskan',
 'georges',
 'at',
 'the',
 'cove',
 'we',
 'enjoyed',
 'this',
 'when',
 'we',
 'visited',
 'this',
 'restaurant',
 'in',
 'la',
 'jolla',
 'california',
 'this',
 'recipe',
 'is',
 'requested',
 'so',
 'often',
 'they',
 'have',
 'it',
 'printed',
 'and',
 'ready',
 'at',
 'the',
 'hostess',
 'stand',
 'its',
 'unbeatable',
 'at',
 'the',
 'restaurant',
 'but',
 'i',
 'do',
 'a',
 'pretty',
 'good',
 'job',
 'at',
 'home',
 'too',
 'if',
 'i',
 'do',
 'say',
 'so',
 'myself',
 'my',
 'children',
 'and',
 'their',
 'friends',
 'ask',
 'for',
 'my',
 'homemade',
 'popsicles',
 'morning',
 'noon',
 'and',
 'night',
 'i',
 'never',
 'turn',
 'them',
 'down',
 'who',
 'am',
 'i',
 'to',
 'tell',
 'them',
 'that',
 'they',
 'are',
 'good',
 'for',
 'them',
 'for',
 'variety',
 'i',
 'substitute',
 'different',
 'flavours',
 'of',
 'frozen',
 'juice',
 'grape',
 'fruit',
 'punch',
 'tropical',
 'etc',
 'these',


In [38]:
sum([i in stopwrds for i in tokenized]) / len(tokenized)

0.45682781101281494

In [41]:
from collections import Counter

c = Counter(tokenized)
print('До')
print(c.most_common(10))

c = Counter(filter(lambda x: x not in stopwrds, tokenized))
print('После')
print(c.most_common(10))

До
[('the', 40210), ('a', 34994), ('and', 30279), ('this', 27048), ('i', 25111), ('to', 23499), ('is', 20290), ('it', 19863), ('of', 18372), ('for', 15988)]
После
[('recipe', 14957), ('make', 6353), ('time', 5180), ('use', 4635), ('great', 4453), ('like', 4175), ('easy', 4175), ('one', 3886), ('good', 3820), ('made', 3814)]


### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer

sample = descriptions.sample(5)
vec = TfidfVectorizer()
res = vec.fit_transform(sample.preprocessed_descriptions).toarray()

3.2 Вычислите близость между каждой парой рецептов, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

In [43]:
from scipy.spatial.distance import cosine

df = pd.DataFrame(index=sample.name.to_list(), columns=sample.name.to_list())
for i in range(len(res)):
    df.iloc[i, i] = 0
    for j in range(i + 1, len(res)):
        dist = cosine(res[i], res[j])
        df.iloc[i, j] = dist
        df.iloc[j, i] = dist
df

,erishte with spicy meats,venison pressure canned,braised fennel and onion pasta sauce,amazing homemade meatballs,rhode island hot wiener sauce 1
erishte with spicy meats,0,0.967604,0.901855,0.836952,0.885993
venison pressure canned,0.967604,0,0.923782,0.918276,0.925808
braised fennel and onion pasta sauce,0.901855,0.923782,0,0.891979,0.877328
amazing homemade meatballs,0.836952,0.918276,0.891979,0,0.862057
rhode island hot wiener sauce 1,0.885993,0.925808,0.877328,0.862057,0


3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат

Наиболее похожими являются `venison   pressure canned` и `erishte with spicy meats`

In [44]:
sample

,Unnamed: 0,name,preprocessed_descriptions
10602,10602,erishte with spicy meats,erishte is turkish handmade macaroni and this ...
28804,28804,venison pressure canned,a great way to have cooked venison in your pan...
3601,3601,braised fennel and onion pasta sauce,fennel is a delightful veggie tasting like lic...
634,634,amazing homemade meatballs,i saw a few recipes and tried to combine some ...
22461,22461,rhode island hot wiener sauce 1,every state has its own food specialties hawai...


erishte is turkish handmade macaroni and this is my private recipe which is exotic and delicious

a great way to have cooked venison in your pantry ready to heat  eat great flavor  it makes its own juicegravy this recipe was given to me by a friend requires a pressure canner